# Modelo de Aprendizaje no Superviasado
# Modelos de detección de anomalías

## Dataset: Credit Card Fraud Detection
## Objetivo: Entrenar un modelo que detecte si una operación es fraudulenta
## Fuente: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
Índice<a name="indice"></a>
* [1 Prepocesado de datos](#1)
    * [1.1 Importación de librerías](#1_1)
    * [1.2 Carga del Dataset](#1_2)
* [2 Análisis Exploratorio de datos](#2)
    * [2.1 Estadísitcas descriptivas](#2_1)
* [3 Diseño de características](#3)
    * [3.1 Tratamiento de datos nulos](#3_1)
    * [3.2 Transformación de características](#3_2)
    * [3.3 Selección de características relevante](#3_3)
    * [3.4 Tratamiento de outliers](#3_4)
    * [3.5 Tratamiento de correlación](#3_5)
    * [3.6 Normalización](#3_6)
* [4 Local Outlier Factor (LOF)](#4) 
* [5 Isolation Tree](#5)


### Descripción de la fuente
Cuenta con un total de 30 variables predictoras X y una variable continua a predecir Y. El número total de muestras es de 284.807 transacciones.
El dataset contiene transacciones de tarjetas realizadas en septiembre de 2013 por entidades europeas. Estas transacciones ocurrieron en dos días, produciéndose un total de 492 transacciones fraudulentas de un total de 284.807. 

Todas las variables en este dataset son numericas. Dos de ellas son 'Time' y 'Amount', las cuales son variables originales. El resto, son resultado de una transformación PCA, siendo V1-V28 las componentes principales. Por motivos de confidencialidad no se proporcionan las variables originales ni más información sobre los datos.

* _Time_: segundos entre cada transacción y la primera transacción del dataset
* _Variables 'V1-V28'_: componentes principales de una transformación de variables utilizando PCA
* _Amount_: unidades monetarias de la transacción
* _Class_: Variable objetivo, indica si esa transacción es fraudulenta (1) o no es fraudulenta (0)

# 1 PREPRECESADO DE DATOS <a name="1"></a>
[Índice](#indice)

## 1.1 Importación de librerías <a name="1_1"></a>
[Índice](#indice)

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns;
import matplotlib.pyplot as plt





## 1.2 Carga del dataset <a name="1_2"></a>
[Índice](#indice)

In [2]:
df = pd.read_csv('creditcard.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [ ]:
df.shape

In [ ]:
# Balanceo
df['Class'].value_counts()

In [ ]:
# Balanceo
df['Class'].value_counts()/len(df)

# 2 PREPRECESADO DE DATOS <a name="2"></a>
[Índice](#indice)

## 2.1 Estadísticas descriptivas <a name="2_1"></a>
[Índice](#indice)

In [ ]:
df.describe()

# 3 DISEÑO DE CARACTERÍSTICAS <a name="3"></a>
[Índice](#indice)

## 3.1 Tratamiento de nulos <a name="3_1"></a>
[Índice](#indice)

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.isin(['','?',' ']).sum()

## 3.2 Transformación de características <a name="3_2"></a>
[Índice](#indice)

In [ ]:
# Transformaciones de tipo

In [ ]:
# No es necesario hacer label encoding

In [ ]:
# One hot encoding

In [ ]:
# Seleccionamos las columnas generadas con OHE y las cambiamos a categóricas

## 3.3 Selección de características relevantes <a name="3_3"></a>
[Índice](#indice)

In [ ]:
# Eliminar irrelevantes
df.drop(['Time'],axis=1,inplace=True)

## 3.4 Tratamiento de outliers <a name="3_4"></a>
[Índice](#indice)

In [ ]:
# Ver en una única gráfica después de normalizar
# Tratamiento de outliers
plt.subplots(sharex=False, sharey=False,figsize=(20,15))
for i in range(len(df.columns)):
    ax = plt.subplot(1,len(df.columns),i+1)
    sns.boxplot(data=df.iloc[:,i],ax=ax)
    ax.set_title(df.columns[i])

## 3.5 Tratamiento de Correlación <a name="3_5"></a>
[Índice](#indice)

In [ ]:
# Tratamiento de correlación
sns.pairplot(data=df,hue='Class',palette='bright')

In [ ]:
sns.heatmap(df.corr(),cmap='coolwarm',annot=True)

## 3.6 Normalización <a name="3_6"></a>
[Índice](#indice)

In [ ]:
# Normalización
from scipy import stats
# Mostrar si las variables siguen una distribución normal según saphiro test
corte = 0.05
for c in df:    
    #Hacemos el saphiroTest
    stat,pvalor=stats.shapiro(df[c])
    #print(f"Variable:{c}\tEstadístico:{stat}\tP-valor:{pvalor}")
    if(pvalor>corte):            
        print(f"Variable {c} provine distribución normal p-valor>0.05\t:{pvalor}")
    else:            
        print(f"Variable {c} no proviene distribución  p-valor<=0.05\t:{pvalor}")

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
dfNorm = sc.fit_transform(df)
dfNorm= pd.DataFrame(dfNorm,columns=df.columns)
dfNorm

# 4 Local Outlier Factor (LOF) <a name="4"></a>
[Índice](#indice)

# 5 Isolation Tree <a name="5"></a>
[Índice](#indice)